### Load the model and make prediction
##### Input Args:

doc_dir = directory where the file with comments to be predicted

doc2vec_model_name = doc2vec model already trained

trained_clf = Trained classifier


In [1]:
from gensim.models import Doc2Vec
import re
import os
import random
from sklearn import metrics
import numpy as np
from sklearn.externals import joblib
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) #strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remoce apostrophes
    sent = re.sub(r'\W',' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated space
    sent = sent.strip()
    return sent.split()

In [4]:
#Load doc to vec model
doc2vec_model_name = 'reviews.d2v'
doc2vec_model = Doc2Vec.load(doc2vec_model_name)

In [5]:
#Test the trained doc2vec model
cosine_similarity(
[doc2vec_model.infer_vector(extract_words("This is very bad video. I don't like it"))],
[doc2vec_model.infer_vector(extract_words("video sucks."))])

array([[0.6934385]], dtype=float32)

In [6]:
#Converts the comments to vector using doc2vec model trianed earlier
def get_doc2vec(model, doc_dir,comment_text_pos=0):
    sentences = []
    sentvecs = []
    
    for fname in sorted(os.listdir(doc_dir)):
        with open(doc_dir + "/" + fname, encoding='UTF-8') as f:
            print("files being read : {0}".format(fname))
            for i, line in enumerate(f):
                line_split = line.strip().split('\t')
                sentences.append(line_split[comment_text_pos])
                words = extract_words(line_split[comment_text_pos])
                sentvecs.append(model.infer_vector(words, steps=10)) # create a vector for this document
        
    #sentences, sentvecs together
    return sentences, sentvecs

In [7]:
doc_dir = "data/Test"
sentences, sentvecs = get_doc2vec(doc2vec_model, doc_dir)

files being read : yelp_labelled.txt


In [8]:
print("sentences : {0}".format(len(sentences)))

sentences : 1000


In [9]:
sentences[0:5]

['Wow... Loved this place.',
 'Crust is not good.',
 'Not tasty and the texture was just nasty.',
 'Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.',
 'The selection on the menu was great and so were the prices.']

In [10]:
sentvecs[0:5]

[array([-0.24903746, -0.18018925,  0.05749322, -0.10560571, -0.33776838,
         0.16579776,  0.17553787,  0.17636697, -0.07393146, -0.14072871,
         0.09156888,  0.02022403, -0.09713821,  0.13383433, -0.17543945,
         0.15709971, -0.03591085,  0.15174581,  0.13046418, -0.01549064,
         0.13690972,  0.2751294 , -0.05023609,  0.15681918, -0.1916324 ,
         0.2513422 , -0.23718649,  0.10328327, -0.14861111, -0.03345313,
        -0.08967361,  0.1354041 , -0.18289335, -0.15940091, -0.2080593 ,
         0.50230086,  0.30272844,  0.18101959,  0.22926323,  0.02978012,
         0.15447672, -0.04651478, -0.01549169,  0.23443107,  0.00571839,
         0.20836784, -0.07697911,  0.03664642,  0.38524595, -0.04609485],
       dtype=float32),
 array([-0.07684638,  0.09519014,  0.05153202, -0.14022346,  0.08552711,
         0.26134858,  0.16231316,  0.05601692,  0.08603787,  0.00941966,
        -0.04827184, -0.14852993, -0.05336305,  0.10257004,  0.11232131,
         0.30058882,  0.025

In [11]:
trained_clf = 'doc2vec_rf.model'
loaded_clf = joblib.load(trained_clf)

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [12]:
y_pred = loaded_clf.predict(sentvecs)
print(y_pred)

[1 1 0 1 0 0 1 0 1 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 1
 0 1 0 1 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0
 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1
 0 1 1 1 1 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 0 0 0 1 1 0 0
 0 0 1 1 0 0 1 1 1 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 0 0 1
 1 0 1 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0
 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1
 1 1 0 1 1 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 0
 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 0 1
 0 1 1 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0
 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1
 1 1 0 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0
 1 1 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0 1 0 0 0 1 1 1 1 0
 1 0 1 0 1 1 0 0 1 1 1 0 

In [13]:
trained_clf = 'doc2vec_kn.model'
loaded_clf = joblib.load(trained_clf)

In [14]:
y_pred = loaded_clf.predict(sentvecs)
print(y_pred)

[1 0 1 1 1 0 0 0 1 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0
 0 1 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 0
 0 1 0 1 1 1 1 0 0 1 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 0 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1
 1 0 1 1 0 0 1 1 1 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1
 1 0 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0
 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 0
 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1 0 1 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1
 0 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 1
 0 1 1 0 0 1 0 0 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1
 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1
 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 1 1 0
 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1 1 0
 0 1 1 0 1 1 1 0 1 0 1 0 